Ucitavamo podatke i pregledavamo njihovu strukturu i velicinu.

In [44]:
import pandas as pd

data = pd.read_csv('data.csv')

print(data.head())
print(data.shape)

                                                text
0  Oh, how the headlines blared:\nChatbots were T...
1  If you’ve ever found yourself looking up the s...
2  Machine learning is increasingly moving from h...
3  If your understanding of A.I. and Machine Lear...
4  Want to learn about applied Artificial Intelli...
(337, 1)


- Format je tipa string (object), postoji jedna kolona, velicina je 337 zapisa.

Nakon uvida u strukturu skupa podataka prelazimo na izvrsavanje transformacije podataka:
- prvo preuzimamo potrebne *korpuse* i *model*
- **tokenizacija** - podjela teksta na dijelove
- **stopwords** - uklanjanje zaustavnih rijeci koje nemaju znacajnu vrijednost(veznici,pomocni glagoli itd...)
- **lematizacija** - proces skracivanja rijeci za smanjenje šuma i poboljsanje pretrage
- **etiketiranje teksta** - dodavaju se oznake ili etikete dijelovima teksta (imenica,glagol itd...)

Cijeli proces ce se izvrsiti u jednoj funkciji

In [45]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

# pozivamo klasu za lematizaciju
lemmatizer = WordNetLemmatizer()
# ucitavanje skupa zaustavnih rijeci
stop_words = set(stopwords.words('english'))

def transform_data(column):
    
    tokens = word_tokenize(column)
    
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    pos_tags = pos_tag(lemmatized_tokens)
    return pos_tags

# pozivamo funkciju transform
# pomocu funkcije apply() da se primjeni na cijeloj koloni
data['processed'] = data['text'].apply(transform_data)

print(data['processed'].head())

0    [(Oh, UH), (,, ,), (headline, JJ), (blared, VB...
1    [(’, NN), (ever, RB), (found, VBD), (looking, ...
2    [(Machine, NN), (learning, VBG), (increasingly...
3    [(understanding, VBG), (A.I, NNP), (., .), (Ma...
4    [(Want, NNP), (learn, NN), (applied, VBD), (Ar...
Name: processed, dtype: object


Dobili smo transformiranu kolonu sa tokenima i njihovim oznakama/etiketama.

Prelazimo na dobijanje *vektora znacenja* - proces *vektorizacije* (pretvaranje teksta u numericki oblik) sa mnogo manje dimenzija i ocuvanja semantickog znacenja rijeci.

Koristit cemo **word2vec** algoritam iz biblioteke *gensim*

Prije svega kreiramo listu i prolazimo kroz 'processed' kolonu.Svaki artikl kolone je lista torki koja sadrzi rijec i odredenu oznaku.U ugniježđenoj listi prolazimo kroz svaku torku, ubacujemo rijec i izbacujemo oznaku zato sto **Word2Vec** model uci kontekstu u kojem se rijeci pojavljuju. Unos oznaka/etiketa zakompliciralo bi proces treniranja modela.

Za pronalazak dvije najslicnije rijeci i dvije sa najmanje slicnim znacenjem koristimo svojstvo **wv** i metoda **most_similar()**
- nakon treiranja modela kreira se objekt **wv** koji sadrzi mapiranja stvarnih rijeci i njihovih vektora znacenja.
- **most_similar** metoda za pronalazenje slicnih rijeci - prosljeduje se zadana rijec i vrijednost parametra **topn** koji definira koliko slicnih rijeci cemo dobiti, u ovom slucaju je dvije rijeci. Za suprotnost zadane rijeci koristimo **negative** parametar.

In [46]:
from gensim.models import Word2Vec

filtrirane_rijeci = [[word for word, tag in article] for article in data['processed']]
# print(rijeci[0])

# treniramo model
model = Word2Vec(filtrirane_rijeci, vector_size=100, window=5, min_count=1, workers=4)

# zadana rijec 
rijec = 'android' 
most_similar = model.wv.most_similar(rijec, topn=2)
least_similar = model.wv.most_similar(negative=[rijec], topn=2)


print("Najslicnija rijec zadanoj rijeci '{}':".format(rijec), most_similar)
print("Najmanje slica rijec zadanoj rijeci '{}':".format(rijec), least_similar)

Najslicnija rijec zadanoj rijeci 'android': [('mistrustful', 0.8223909735679626), ('souvent', 0.8174886107444763)]
Najmanje slica rijec zadanoj rijeci 'android': [('restate', 0.5823929309844971), ('3d-geometry', 0.5622067451477051)]
